In [0]:
spark.range(5).show()

In [0]:
!pip install kaggle

In [0]:
import os

os.environ["KAGGLE_USERNAME"] = "aj11111jajoo"
os.environ["KAGGLE_KEY"] = "KGAT_6cf8b5a04da516c6a2e5abf5329508fd"

print("Kaggle credentials configured!")

In [0]:
spark.sql("""
CREATE SCHEMA IF NOT EXISTS workspace.ecommerce
""")

In [0]:
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.ecommerce_data
""")

In [0]:
%sh
mkdir -p /Volumes/workspace/ecommerce/ecommerce_data
cd /Volumes/workspace/ecommerce/ecommerce_data
kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
unzip -o ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
rm -f ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

In [0]:
%restart_python

In [0]:
df_n = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv")

In [0]:
df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv")

In [0]:
print(f"October 2019 - Total Events: {df.count():,}")
print("\n" + "="*60)
print("SCHEMA:")
print("="*60)
df.printSchema()

In [0]:
print("\n" + "="*60)
print("SAMPLE DATA (First 5 rows):")
print("="*60)
df.show(5, truncate=False)

In [0]:
spark.sql("CREATE CATALOG IF NOT EXISTS workspace")
spark.sql("CREATE SCHEMA IF NOT EXISTS workspace.ecommerce")
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.data_volume
""")

In [0]:
path = "/Volumes/workspace/ecommerce/ecommerce_data/*.csv"

df = (spark.read
      .option("header", True)
      .option("inferSchema", True)
      .csv(path))

# Use sample for learning to avoid heavy load
df_sample = df.sample(0.02)

df_sample.printSchema()
df_sample.count()

In [0]:
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.ecommerce_delta
""")

In [0]:
spark.sql("SHOW VOLUMES IN workspace.ecommerce").show()

In [0]:
delta_path = "/Volumes/workspace/ecommerce/ecommerce_delta"

(df_sample.write
 .format("delta")
 .mode("overwrite")
 .save(delta_path))

In [0]:
spark.read.format("delta").load(delta_path).count()

In [0]:
delta_path = "/Volumes/workspace/ecommerce/ecommerce_delta"

spark.sql("DROP TABLE IF EXISTS workspace.ecommerce.ecommerce_delta_table")

spark.read.format("delta").load(delta_path) \
     .write.format("delta") \
     .mode("overwrite") \
     .saveAsTable("workspace.ecommerce.ecommerce_delta_table")

In [0]:
spark.sql("SELECT COUNT(*) FROM workspace.ecommerce.ecommerce_delta_table").show()

In [0]:
for i in range(5):
    (df_sample.limit(10000)
     .write
     .format("delta")
     .mode("append")
     .save(delta_path))

In [0]:
spark.sql("""
DESCRIBE DETAIL workspace.ecommerce.ecommerce_delta_table
""").select("numFiles","sizeInBytes").show(truncate=False)

In [0]:
spark.sql("""
OPTIMIZE workspace.ecommerce.ecommerce_delta_table
""")

In [0]:
spark.sql("""
DESCRIBE DETAIL workspace.ecommerce.ecommerce_delta_table
""").select("numFiles","sizeInBytes").show(truncate=False)